In [ ]:
!pip install gensim
import os
os._exit(0)

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
pip install -U langchain-community neo4j flair langchain_openai PyMuPDF pdfplumber pytesseract

In [ ]:
import os
import gc
from bs4 import BeautifulSoup
import re
from langchain.document_loaders import TextLoader
from langchain.schema import Document
import numpy as np
import logging
from typing import List
import time
import flair
from flair.data import Sentence
from flair.models import SequenceTagger
from neo4j import GraphDatabase
from langchain.text_splitter import RecursiveCharacterTextSplitter
from concurrent.futures import ProcessPoolExecutor
import fitz
import pytesseract
import pdfplumber
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
!apt-get install tesseract-ocr

In [ ]:
# Setup flair model for entity extraction
tagger = SequenceTagger.load("flair/ner-english-ontonotes-large")

In [ ]:
# Setup API key
API_KEY = 'YOUR_API_KEY'
os.environ['OPENAI_API_KEY'] = API_KEY

In [ ]:
# Setup LLM for relationship
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

In [ ]:
# Setup neo4j graph database driver
URI = "YOUR_URI"
AUTH = ("neo4j", "YOUR_PASSWORD")
driver = GraphDatabase.driver(URI, auth=AUTH)

In [ ]:
# For metadata and relation consistency
nasdaq_100_mapping = {
    "AAPL": "Apple Inc.",
    "MSFT": "Microsoft Corporation",
    "AMZN": "Amazon.com, Inc.",
    "GOOG": "Alphabet Inc. (Class C)",
    "GOOGL": "Alphabet Inc. (Class A)",
    "META": "Meta Platforms, Inc.",
    "NVDA": "NVIDIA Corporation",
    "TSLA": "Tesla, Inc.",
    "PEP": "PepsiCo, Inc.",
    "COST": "Costco Wholesale Corporation",
    "ADBE": "Adobe Inc.",
    "CMCSA": "Comcast Corporation",
    "CSCO": "Cisco Systems, Inc.",
    "AVGO": "Broadcom Inc.",
    "INTC": "Intel Corporation",
    "NFLX": "Netflix, Inc.",
    "TXN": "Texas Instruments Incorporated",
    "QCOM": "QUALCOMM Incorporated",
    "INTU": "Intuit Inc.",
    "AMGN": "Amgen Inc.",
    "SBUX": "Starbucks Corporation",
    "ISRG": "Intuitive Surgical, Inc.",
    "MDLZ": "Mondelez International, Inc.",
    "BKNG": "Booking Holdings Inc.",
    "ADI": "Analog Devices, Inc.",
    "PYPL": "PayPal Holdings, Inc.",
    "ADP": "Automatic Data Processing, Inc.",
    "LRCX": "Lam Research Corporation",
    "GILD": "Gilead Sciences, Inc.",
    "FISV": "Fiserv, Inc.",
    "VRTX": "Vertex Pharmaceuticals Incorporated",
    "REGN": "Regeneron Pharmaceuticals, Inc.",
    "KDP": "Keurig Dr Pepper Inc.",
    "ASML": "ASML Holding N.V.",
    "ORLY": "O'Reilly Automotive, Inc.",
    "MNST": "Monster Beverage Corporation",
    "MELI": "MercadoLibre, Inc.",
    "CTSH": "Cognizant Technology Solutions Corporation",
    "SNPS": "Synopsys, Inc.",
    "PANW": "Palo Alto Networks, Inc.",
    "CDNS": "Cadence Design Systems, Inc.",
    "MAR": "Marriott International, Inc.",
    "ROST": "Ross Stores, Inc.",
    "IDXX": "IDEXX Laboratories, Inc.",
    "EXC": "Exelon Corporation",
    "CSX": "CSX Corporation",
    "MCHP": "Microchip Technology Incorporated",
    "AEP": "American Electric Power Company, Inc.",
    "XEL": "Xcel Energy Inc.",
    "DLTR": "Dollar Tree, Inc.",
    "PAYX": "Paychex, Inc.",
    "WBA": "Walgreens Boots Alliance, Inc.",
    "CTAS": "Cintas Corporation",
    "PCAR": "PACCAR Inc",
    "ODFL": "Old Dominion Freight Line, Inc.",
    "FAST": "Fastenal Company",
    "ANSS": "ANSYS, Inc.",
    "VRSK": "Verisk Analytics, Inc.",
    "BIIB": "Biogen Inc.",
    "CHTR": "Charter Communications, Inc.",
    "EA": "Electronic Arts Inc.",
    "SIRI": "Sirius XM Holdings Inc.",
    "ILMN": "Illumina, Inc.",
    "WBD": "Warner Bros. Discovery, Inc.",
    "ZM": "Zoom Video Communications, Inc.",
    "KHC": "The Kraft Heinz Company",
    "ALGN": "Align Technology, Inc.",
    "CEG": "Constellation Energy Corporation",
    "DDOG": "Datadog, Inc.",
    "DOCU": "DocuSign, Inc.",
    "EBAY": "eBay Inc.",
    "ENPH": "Enphase Energy, Inc.",
    "FANG": "Diamondback Energy, Inc.",
    "FTNT": "Fortinet, Inc.",
    "GEHC": "GE HealthCare Technologies Inc.",
    "GFS": "GLOBALFOUNDRIES Inc.",
    "HON": "Honeywell International Inc.",
    "KLAC": "KLA Corporation",
    "LULU": "Lululemon Athletica Inc.",
    "MSTR": "MicroStrategy Incorporated",
    "MU": "Micron Technology, Inc.",
    "NXPI": "NXP Semiconductors N.V.",
    "ON": "ON Semiconductor Corporation",
    "PANW": "Palo Alto Networks, Inc.",
    "PDD": "Pinduoduo Inc.",
    "PLTR": "Palantir Technologies Inc.",
    "ROK": "Rockwell Automation, Inc.",
    "SGEN": "Seagen Inc.",
    "SPLK": "Splunk Inc.",
    "TEAM": "Atlassian Corporation",
    "TTD": "The Trade Desk, Inc.",
    "VRSN": "VeriSign, Inc.",
    "WDAY": "Workday, Inc.",
    "ZS": "Zscaler, Inc."
}

**Load documents**

In [ ]:
# PDF Document Directory
PDF_PATH = '/content/drive/MyDrive/Colab Notebooks/transcripts/'

In [ ]:
class Document:
  def __init__(self, page_content: str, metadata: dict = None):
    self.page_content = page_content
    self.metadata = metadata if metadata else {}

class PDFLoader(TextLoader):
  def __init__(self, file_path):
    self.file_path = file_path
    self.metadata = self.extract_metadata(file_path)

  def extract_metadata(self, file_path: str) -> dict:
    company_ticker = os.path.basename(os.path.dirname(file_path))
    return {'source': 'Transcript', 'company_ticker': company_ticker}

  def extract_text_from_pdf(self, pdf_path: str) -> str:
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text")
    return text

  def extract_text_from_image_pdf(self, pdf_path: str) -> str:
    text = ""
    try:
      with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
          pix = page.to_image()
          img = pix.original
          text += pytesseract.image_to_string(img)  # OCR extraction
    except Exception as e:
      print(f"Error extracting text from {pdf_path}: {e}")
    return text

  def preprocess(self, text: str, metadata: dict) -> List[Document]:
    # Returns a list of Document objects
    clean_text = clean_text(text)
    return [Document(page_content=clean_text, metadata=metadata)]

  def clean_text(self, text: str) -> str:
    patterns = [
            (r'\b\d{8,}\b', ''),
            (r'\b\d{2,4}[-/\.\d]*\b', ''),
            (r'\s+', ' '),
            (r'[^a-zA-Z0-9\s.,!?\'"(){}-]', ''),
        ]
    for pattern, replacement in patterns:
      text = re.sub(pattern, replacement, text)
    return text.strip()


In [ ]:
def load(filepath) -> List[tuple] :
  files = []
  for root, dirs, files_in_dir in os.walk(filepath):
    for file_name in files_in_dir:
      if file_name.lower().endswith('.pdf'):
        company_ticker = os.path.basename(root)
        file_path = os.path.join(root, file_name)
        files.append((file_path, company_ticker))
  return files

In [ ]:
def load_all_docs(file_path: str) -> List[Document]:
  pdf_loader = PDFLoader(file_path)
  pdf_files = load(file_path)  # This returns a list of tuples (file_path, company_ticker)

  all_docs = []
  for pdf_file, company_ticker in pdf_files:
    print(f"Processing {pdf_file}...")

    # Extract metadata for the document
    metadata = pdf_loader.extract_metadata(pdf_file)

    # Try to extract text normally first (text-based PDFs)
    text = pdf_loader.extract_text_from_pdf(pdf_file)

    # If no text was extracted, use OCR (image-based PDFs)
    if not text.strip():
      print(f"Using OCR for {pdf_file}...")
      text = pdf_loader.extract_text_from_image_pdf(pdf_file)

    # Preprocess the extracted text and metadata into Document objects
    documents = pdf_loader.preprocess(text, metadata)

    # Add all the processed Document objects to the all_docs list
    all_docs.extend(documents)

  return all_docs

In [ ]:
all_docs = load_all_docs(PDF_PATH)

**Split doc into chunks**

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ","],
    add_start_index = True
)
split_documents = text_splitter.split_documents(all_docs)

**Extract entities from text within chunks**

In [ ]:
# FINANCIAL_TERMS not applicable to our current usage
# FINANCIAL_TERMS = {
#     "Revenue": ["Revenue", "Net Sales"],
#     "EBITDA": ["EBITDA"],
#     "FCF": ["Free Cash Flow"],
#     "Net Income": ["Net Income", "Net Profit"],
#     "Operating Income": ["Operating Income", "Operating Profit"],
#     "Gross Profit": ["Gross Profit"],
#     'ACV': ['ACV']
#     }
EXECUTIVE_TITLES = ["CEO", "Chairman", "President", "Director"]
PRODUCT_PATTERNS = [
    r"our (?:products|services) include:?\s*(.*?)[\.;]",  # "Our products include: []."
    r"we offer\s*(.*?)[\.;]",  # "We offer []."
    r"we provide\s*(.*?)[\.;]",  # "We provide [] services."
    ]
GEO_PATTERNS = {
    "Headquarters": r"headquartered in ([A-Za-z\s,]+)",
    "Operating Regions": r"operate in ([A-Za-z\s,]+)",
    "Markets": r"market[s]? include[s]? ([A-Za-z\s,]+)",
}
EVENT_PATTERNS = {
    "Mergers & Acquisitions": r"acquired\s+([A-Za-z\s]+)\s+for\s+\$?[\d,.]+",
    "Product Launches": r"launched\s+([A-Za-z\s]+)\s+in\s+\d{4}",
    "Earnings Calls": r"our\s+Q\d\s+earnings\s+call",
}
LEGAL_PATTERNS = {
    "Legal Cases": r"(lawsuit|litigation|antitrust case|settlement)"
}

In [ ]:
# For financial data, not applicable to our current usage
UNIT_CONVERSION = {
    "billion": 1000,  # Billion → Convert to million
    "million": 1,  # Million
    "thousand": 0.001  # Thousand → Convert to million
}
def normalize_number(value, unit):
  """Convert financial numbers into standardized millions."""
  num = float(value.replace(",", ""))  # Remove commas
  multiplier = UNIT_CONVERSION.get(unit.lower(), 1)  # Default to million if missing
  return round(num * multiplier, 3)  # Keep 3 decimal places

In [ ]:
def extract_entities(text):

  sentence = Sentence(text)
  tagger.predict(sentence)

  # # For financial data, not applicable to our current usage
  # extracted = {key: [] for key in FINANCIAL_TERMS}
  extracted={
      "Company": [],
      "Executives": [],
      "Products": [],
      "Geo": {"Headquarters": [], "Operating Regions": [], "Markets": []},
      "Events": {"Mergers & Acquisitions": [], "Product Launches": [], "Earnings Calls": []},
      "Legal Issues": {"Legal Cases": []},
    }

  detected_people = []
  detected_title = []
  detected_products = []
  detected_geo = []
  detected_legal = []

  for entity in sentence.get_spans("ner"):
    if entity.tag == "ORG":  # Company names
      if entity.text != 'Company':
        extracted["Company"].append(entity.text)
    # # For financial data, not applicable to our current usage
    # elif entity.tag == "MONEY":  # Financial numbers (Revenue, Market Cap)
    #   money_value = entity.text.replace("$", "").replace(",", "").strip()
    #   if money_value.isnumeric():  # Ensure it's a valid number
    #     extracted.setdefault("MONEY", []).append(money_value)
    elif entity.tag in ["PRODUCT", "WORK_OF_ART"]:
      detected_products.append(entity.text)
    elif entity.tag == "GPE":
      detected_geo.append(entity.text)
    elif entity.tag == "PERSON":  # Executives
      detected_people.append(entity.text)
    elif entity.tag == "LAW":  # Legal references
      detected_legal.append(entity.text)

  # For financial data, not applicable to our current usage
  # for key, variations in FINANCIAL_TERMS.items():
  #   for term in variations:
  #     match = re.search(rf"({term})[^$]*\$\s?([\d,.]+)\s*(billion|million|thousand)?", text, re.IGNORECASE)
  #     if match:
  #       value, unit = match.group(2), match.group(3)
  #       extracted[key].append(str(normalize_number(value, unit)) + " million")

  # Match Executive
  for title in EXECUTIVE_TITLES:
    match = re.search(rf"([\w\s]+)\s+is\s+(?:the\s+)?{title}\s+of\s+([\w\s]+)|{title}[:\-]?\s*([\w\s]+)\s*\(?([\w\s]*)\)?", text, re.IGNORECASE)
    if match:
      detected_title.append((match.group(1), match.group(2)))

  # Extract executive and title
  for title, name in detected_title:
    # Ensure the detected name is in NER-detected people list
    if name in detected_people or len(detected_people) == 1:
      extracted["Executives"].append(f"{title}: {name}")
    elif len(detected_people) > 1:
      extracted["Executives"].append(f"{title}: {detected_people.pop(0)}")

  # Extract Geo
  for key, pattern in GEO_PATTERNS.items():
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
      extracted["Geo"][key].append(match.group(1).strip())

  # Extract Events
  for key, pattern in EVENT_PATTERNS.items():
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
      extracted["Events"][key].append(match.group(1).strip() if key != "Earnings Calls" else "Detected")

  # Extract Legal Issues
  for key, pattern in LEGAL_PATTERNS.items():
    if re.search(pattern, text, re.IGNORECASE):
      extracted["Legal Issues"][key].append("Detected")

  # Extract Products
  for pattern in PRODUCT_PATTERNS:
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
      extracted["Products"].extend([item.strip() for item in match.group(1).split(",")])

  extracted["Geo"]["Headquarters"].extend(detected_geo)
  extracted["Legal Issues"]["Legal Cases"].extend(detected_legal)

  return extracted

**Use LLM to generate triplet**

In [ ]:
PROMPT_TEMPLATE = """
Generate the necessary data for a triplet that can be directly stored in Neo4j, only from the following extracted entities from a 10-K filing:

Company: {Company}
Executives: {Executives}
Products: {Products}
Geo (Headquarters, Operating Regions, Markets): {Geo}
Events (Mergers & Acquisitions, Product Launches, Earnings Calls): {Events}
Legal Issues (Legal Cases, Regulatory Compliance): {Legal_Issues}

The output of the triplet should be formatted as follows, which could be directly used to store in Neo4j:

(head_entity, relationship, object_entity, entity_category)

For example: ("Microsoft", "IS_THE_CEO_OF", "Satya Nadella", "Executives")

However, if no relationship can be determined from a category, return exactly "No clear relationship" within the categoty of the output.
"""

In [ ]:
# Adjust based on actual situation
EXCLUDED_ENTITIES = {"USGovernmentAgenciesDebtSecuritiesMember", "UNITED STATES SECURITIES",'the Securities and Exchange Commission', 'Investor Relations', 'non-GAAP', 'GAAP', 'ACV'}
EXCLUDED_OBJECTS = {"Executives", "Products", "Geo", "Events", "Legal Issues", "Unknown", "Markets", 'Legal Cases', 'Headquarters', 'Operating Regions', 'Product Launches', 'Earnings Calls', 'Regulatory Compliance', 'Companys', 'The Company', 'Mergers & Acquisitions'}

In [ ]:
def generate_triplet(chunk):
  entities = extract_entities(chunk.page_content)
  metadata = chunk.metadata

  # Fill prompt with data, financial data not applicable to our current usage
  filled_prompt = PROMPT_TEMPLATE.format(
      # Revenue=", ".join(entities.get('Revenue', [])),
      # EBITDA=", ".join(entities.get('EBITDA', [])),
      # FCF=", ".join(entities.get('FCF', [])),
      # Net_Income=", ".join(entities.get('Net Income', [])),
      # Operating_Income=", ".join(entities.get('Operating Income', [])),
      # Gross_Profit=", ".join(entities.get('Gross Profit', [])),
      # ACV=", ".join(entities.get('ACV', [])),
      Company=", ".join(entities.get('Company', [])),
      Executives=", ".join(entities.get('Executives', [])),
      Products=", ".join(entities.get('Products', [])),
      Geo=", ".join([f"{key}: {', '.join(val)}" for key, val in entities.get('Geo', {}).items()]),
      Events=", ".join([f"{key}: {', '.join(val)}" for key, val in entities.get('Events', {}).items()]),
      Legal_Issues=", ".join([f"{key}: {', '.join(val)}" for key, val in entities.get('Legal Issues', {}).items()])
    )

  # triplet generated from LLM
  response = llm(filled_prompt)
  triplet_strings = response.content.strip().split('\n')
  triplets = []

  # Add metadata
  formatted_metadata = ", ".join([f"{key}: {val}" for key, val in metadata.items()])

  for triplet in triplet_strings:
    triplet = triplet.strip("[]").replace('"', '').strip()
    triplet_parts = triplet.split(", ")

    if len(triplet_parts) != 4 or "NO_CLEAR_RELATIONSHIP" in triplet or 'HAS_NO_INFORMATION' in triplet or 'None' in triplet:
      continue

    # Adjust accordingly to prompt
    head_entity, relationship, object_entity, entity_category = triplet_parts

    if head_entity in EXCLUDED_ENTITIES or object_entity in EXCLUDED_ENTITIES:
      continue
    if object_entity == "No clear relationship" or object_entity == "":
      continue
    if object_entity in EXCLUDED_OBJECTS:
      continue
    if head_entity in ['Company', 'Registrant', 'CommonStockMember']:
      ticker = metadata.get('company_ticker', head_entity)
      head_entity = nasdaq_100_mapping.get(ticker, ticker)

    triplets.append((head_entity, relationship, object_entity, entity_category, formatted_metadata))

  return triplets

**Store triplet in neo4j database**

In [ ]:
def store_triplet_in_neo4j(triplet):
  graph = driver.session()
  try:
    metadata_str = triplet[4]
    metadata_dict = dict(item.split(": ", 1) for item in metadata_str.split(", ") if ": " in item)

    head_entity = triplet[0].strip("()")
    relationship = triplet[1].strip('"')
    object_entity = triplet[2].strip('"')
    entity_category = triplet[3].strip('"')

    if relationship.lower() == "no clear relationship" or relationship.lower() == "HAS_NO_INFORMATION":
      return

    if any(term in head_entity for term in ["Company", "Registrant", "CommonStockMember"]):
      ticker = metadata_dict.get('company_ticker', head_entity)
      head_entity = nasdaq_100_mapping.get(ticker, ticker)
    if any(term in object_entity for term in ["Company", "Registrant", "CommonStockMember"]):
      ticker = metadata_dict.get('company_ticker', object_entity)
      object_entity = nasdaq_100_mapping.get(ticker, ticker)
    if head_entity in EXCLUDED_ENTITIES or object_entity in EXCLUDED_ENTITIES:
      return

    with driver.session() as session:
      cypher_query = """
      MERGE (head:Entity {name: $head_entity})
      SET head.category = $category
      """ + "".join([f" SET head.{key} = '{value}'" for key, value in metadata_dict.items()]) + """

      MERGE (object:Entity {name: $object_entity})
      SET object.category = $category

      MERGE (head)-[r:""" + relationship + """]->(object)
      """

      session.run(
          cypher_query,
          head_entity=head_entity,
          object_entity=object_entity,
          category=entity_category
          )

      print(f"Stored triplet: ({head_entity}) -[{relationship}]-> ({object_entity}) with metadata {metadata_dict}")

  except Exception as e:
    print(f"Error while storing triplet: {triplet}. Error: {e}")

In [ ]:
for doc in split_documents:
  triplets = generate_triplet(doc)
  for triplet in triplets:
    store_triplet_in_neo4j(triplet)